<a href="https://colab.research.google.com/github/Kaguya2906/Chicago-City-Crime-Data-Retrieval-Platform/blob/main/03.%20Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 Create and populate MongoDB database

## Initialize MongoDB client and database

In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
db = client.apan5400

## Data Cleaning

In [ ]:
df = df.drop(['IUCR','Beat','District','Ward','Community Area','FBI Code','X Coordinate','Y Coordinate'], axis=1)
df = df.dropna()
df.to_csv('Crimes_-_2001_to_Present.csv', index=False)
df.columns
mytype = df['Primary Type'].unique()

#02 Working With JSON File

## Convert to JSON
Convert the DataFrame to a JSON object using the to_json() function, print out the list, and write it to a JSON string(text) file.

In [ ]:
import json
from pprint import pprint
json_list = df.to_json(orient='records')
str_list = json.loads(json_list)

In [ ]:
with open('Crimes_-_2001_to_Present.json', 'w') as f:
    for record in str_list:
        line = json.dumps(record)
        f.write(line)
        f.write("\n")

Import the dataset, Insert multiple JSON documents, Read JSON string file into a Python array of dictionaries

In [ ]:
import json
json_data = open("Crimes_-_2001_to_Present.json", 'r').readlines()
newsfeeds = []
for line in json_data:
    newsfeeds.append(json.loads(line))
print(len(newsfeeds))

In [ ]:
from pprint import pprint
pprint(newsfeeds[0])

Count documents in the collection

In [ ]:
# Up to 2023/04, there are 7760248 data entries
collection.insert_many(newsfeeds)
total_docs = collection.count_documents({})
total_docs

In [ ]:
collection = db.crimes
collection.find_one()

{'_id': ObjectId('64346c6a606e015b4418a9c1'),
 'ID': 10224738,
 'Case Number': 'HY411648',
 'Date': '09/05/2015 01:30:00 PM',
 'Block': '043XX S WOOD ST',
 'Primary Type': 'BATTERY',
 'Description': 'DOMESTIC BATTERY SIMPLE',
 'Location Description': 'RESIDENCE',
 'Arrest': False,
 'Domestic': True,
 'Year': 2015,
 'Updated On': '02/10/2018 03:50:01 PM',
 'Latitude': 41.815117282,
 'Longitude': -87.669999562,
 'Location': '(41.815117282, -87.669999562)'}

# 03 Pipelines with Elasticsearch

In [ ]:
#! pip install Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(["http://localhost:9200"])

In [ ]:
results = collection.find({})
results

In [ ]:
collection.find_one()

In [ ]:
#! pip install tqdm

In [ ]:
# Just have to run once
from tqdm import tqdm
for result in tqdm(results, total=total_docs, desc="Indexing documents"):
    doc = dict(result)
    doc.pop('_id', None)  # remove ObjectId field
    es.index(index="myindex", document=doc)

## Elasticsearch

Below I ask the function to return 10 hits, just to make sure that everything is set up correctly. Later it will be connected to the front-end in https://5008

In [ ]:
def search(query):
    body = {
        "source": ["Date", "Primary Type", "Location Description","Arrest"],
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["Date"]
            }
        },
        "size": 10  # Set size to retrieve up to 10 hits
    }
    response = es.search(body=body)
    return response['hits']['hits']

# print the search results
search_results = search("02/10/2018 01:30:00 PM")
print(pd.DataFrame(search_results))

/var/folders/fh/q6gx0j6d0d5f_5pw7h91mcxc0000gn/T/ipykernel_2659/2446453107.py:12: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(body=body)


    _index _type                   _id    _score  \
0  myindex  _doc  Y8k_bocBFS-c2pFbSy4v  9.951579   
1  myindex  _doc  kMlCbocBFS-c2pFbx7rM  9.951579   
2  myindex  _doc  4MlCbocBFS-c2pFbyrpQ  9.951579   
3  myindex  _doc  0slCbocBFS-c2pFb0bs4  9.951579   
4  myindex  _doc  gMlCbocBFS-c2pFb1ryn  9.951579   
5  myindex  _doc  o8lCbocBFS-c2pFb17yu  9.951579   
6  myindex  _doc  _slCbocBFS-c2pFb2rxt  9.951579   
7  myindex  _doc  JslCbocBFS-c2pFb273j  9.951579   
8  myindex  _doc  T8lCbocBFS-c2pFb3b2V  9.951579   
9  myindex  _doc  D8lCbocBFS-c2pFb476p  9.951579   

                                             _source  
0  {'Primary Type': 'OFFENSE INVOLVING CHILDREN',...  
1  {'Primary Type': 'THEFT', 'Arrest': False, 'Lo...  
2  {'Primary Type': 'BATTERY', 'Arrest': True, 'L...  
3  {'Primary Type': 'THEFT', 'Arrest': False, 'Lo...  
4  {'Primary Type': 'ASSAULT', 'Arrest': False, '...  
5  {'Primary Type': 'THEFT', 'Arrest': False, 'Lo...  
6  {'Primary Type': 'THEFT', 'Arrest': Fal

/var/folders/fh/q6gx0j6d0d5f_5pw7h91mcxc0000gn/T/ipykernel_2659/2446453107.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  response = es.search(body=body)


# 04 Working with Flask

Connect Flask to MongoDB and Elasticsearch in a)HTML Webpage format and b)User-Interface format

## Simple Output

Open a web browser and navigate to http://localhost:5007/ to view the home page of the Flask application.

In [ ]:
from flask import Flask, render_template
app = Flask("MyApp")

@app.route('/')
def hello_world():
    output = "Hey, we are Group 1! This is the homepage of our Flask application. Here we build two searching engine to retrive information of Chicago Crime Reporting."
    return render_template('index.html', result=output)
    #return output

In [ ]:
app.run(host='localhost', port=5007)

 * Serving Flask app "MyApp" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5007/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Apr/2023 21:56:30] "GET / HTTP/1.1" 200 -


## Interactive Output

### Connect Flask to Elasticsearch

http://localhost:5008/

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(["http://localhost:9200"])

from flask import Flask, render_template, request

app = Flask("MyApp")

@app.route('/', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        # Handle POST requests
        query = request.form.get('q')
        if not query:
            return render_template('search.html', results=None)
        else:
            body = {
                "query": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Date", "Primary Type", "Location Description"]
                    }
                },
                "size": 100  # Set size to retrieve up to 10 hits
            }
            results = es.search(index='myindex', body=body)
            return render_template('results.html', query=query, results=results)
    else:
        # Handle GET requests
        return render_template('search.html', results=None)

import os

template_dir = os.path.join(os.getcwd(), 'templates')

if not os.path.exists(template_dir):
    os.makedirs(template_dir)

with open(os.path.join(template_dir, 'results.html'), 'w') as f:
    f.write('''
{% extends "base.html" %}

{% block content %}
  <h1>Search Results for "{{ query }}"</h1>
  {% if results.hits %}
    <table>
      <thead>
        <tr>
          <th>Date</th>
          <th>Primary Type</th>
          <th>Location Description</th>
          <th>Arrest</th>
        </tr>
      </thead>
      <tbody>
        {% for hit in results.hits.hits %}
          {% set source = hit._source %}
          <tr>
            <td>{{ source.Date }}</td>
            <td>{{ source['Primary Type'] }}</td>
            <td>{{ source['Location Description'] }}</td>
            <td>{{ source.Arrest }}</td>
          </tr>
        {% endfor %}
      </tbody>
    </table>
  {% else %}
    <p>No results found.</p>
  {% endif %}
{% endblock %}
''')

with open(os.path.join(template_dir, 'base.html'), 'w') as f:
    f.write('''
<!DOCTYPE html>
<html>
  <head>
    <title>{% block title %}{% endblock %}</title>
    <meta charset="UTF-8">
  </head>
  <body>
    {% block content %}{% endblock %}
  </body>
</html>
''')

if __name__ == '__main__':
    app.run(host='localhost', port=5008)

 * Serving Flask app "MyApp" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5008/ (Press CTRL+C to quit)


### Connect Flask to MongoDB

Since Elasticsearch is a fine-tuned version of MongoDB, we decided to keep the Elasticsearch version, which is neat and fast. I leave the code here for backup.

In [ ]:
from flask import Flask, render_template, request
from pymongo import MongoClient
from tqdm import tqdm

app = Flask("MyApp")
client = MongoClient('mongodb://localhost:27017')
db = client.apan5400
collection = db['crimes']

@app.route('/', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        query = request.form['query']
        results = []
        total_docs = collection.count_documents({})
        for result in tqdm(collection.find({"$text": {"$search": query}}), total=total_docs, desc="Searching documents"):
            doc = dict(result)
            doc.pop('_id', None)  # remove ObjectId field
            results.append(doc)
        return render_template('results.html', results=results, query=query)
    return render_template('search.html')

import os

template_dir = os.path.join(os.getcwd(), 'templates')

if not os.path.exists(template_dir):
    os.makedirs(template_dir)

with open(os.path.join(template_dir, 'results.html'), 'w') as f:
    f.write('''
<!DOCTYPE html>
<html>
  <head>
    <title>Search Results</title>
    <style>
      table {
        width: 100%;
        border-collapse: collapse;
      }
      th, td {
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }
      th {
        background-color: #f2f2f2;
      }
    </style>
  </head>
  <body>
    <h1>Search Results for "{{ query }}"</h1>
    {% if results %}
      <table>
        <tr>
          {% for key in results[0].keys() %}
            <th>{{ key }}</th>
          {% endfor %}
        </tr>
        {% for result in results %}
          <tr>
            {% for value in result.values() %}
              <td>{{ value }}</td>
            {% endfor %}
          </tr>
        {% endfor %}
      </table>
    {% else %}
      <p>No results found.</p >
    {% endif %}
  </body>
</html>

    ''')

if __name__ == '__main__':
    app.run()

 * Serving Flask app "MyApp" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
